# 神经网络增强 RRT 算法教程

本教程展示如何使用神经网络增强 RRT 算法进行路径规划。主要内容包括：

1. 环境准备和依赖安装
2. 训练数据生成和预处理
3. 神经网络模型训练
4. 使用训练好的模型进行路径规划
5. 基于 Pygame 的可视化演示

## 环境准备

首先确保已安装所需的依赖包：

In [ ]:
!pip install torch numpy pygame matplotlib tqdm shapely pyyaml

## 1. 导入必要的模块

In [ ]:
import os
import torch
import numpy as np
from tqdm.notebook import tqdm

from ml.data import DataGenerator, create_data_loaders
from ml.models.rrt_nn import NeuralRRT, SamplingNetwork, EvaluationNetwork, OptimizationNetwork
from simulation.environment import Environment
from rrt.rrt_star import RRTStar

## 2. 训练数据准备

### 2.1 创建数据生成器

In [ ]:
# 配置数据生成器参数
generator = DataGenerator(
    env_width=1000.0,
    env_height=500.0,
    grid_size=(64, 64),
    min_obstacles=20,
    max_obstacles=50,
    num_samples=1000
)

print("数据生成器初始化完成")

### 2.2 生成训练样本

In [ ]:
# 生成训练样本
num_examples = 1000
examples = []

with tqdm(total=num_examples, desc="生成训练样本") as pbar:
    while len(examples) < num_examples:
        example = generator.generate_example()
        if example is not None:
            examples.append(example)
            pbar.update(1)

# 创建数据集
dataset = generator.generate_dataset(examples)

# 创建数据加载器
train_loader, val_loader = create_data_loaders(
    dataset,
    batch_size=32,
    train_ratio=0.8
)

### 2.3 查看数据集统计信息

In [ ]:
def print_dataset_stats(dataset):
    """打印数据集统计信息"""
    print("\n数据集统计信息:")
    print(f"样本数量: {len(dataset)}")
    
    # 计算路径长度统计
    path_lengths = [ex.path_length for ex in dataset.examples]
    print("\n路径长度统计:")
    print(f"  最小值: {min(path_lengths):.2f}")
    print(f"  最大值: {max(path_lengths):.2f}")
    print(f"  平均值: {np.mean(path_lengths):.2f}")
    print(f"  标准差: {np.std(path_lengths):.2f}")
    
    # 计算平滑度统计
    smoothness = [ex.smoothness for ex in dataset.examples]
    print("\n平滑度统计:")
    print(f"  最小值: {min(smoothness):.2f}")
    print(f"  最大值: {max(smoothness):.2f}")
    print(f"  平均值: {np.mean(smoothness):.2f}")
    print(f"  标准差: {np.std(smoothness):.2f}")
    
    # 计算间隙统计
    clearance = [ex.clearance for ex in dataset.examples]
    print("\n障碍物间隙统计:")
    print(f"  最小值: {min(clearance):.2f}")
    print(f"  最大值: {max(clearance):.2f}")
    print(f"  平均值: {np.mean(clearance):.2f}")
    print(f"  标准差: {np.std(clearance):.2f}")

print_dataset_stats(dataset)

## 3. 模型训练

### 3.1 初始化模型

In [ ]:
# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 初始化神经网络
sampling_net = SamplingNetwork().to(device)
evaluation_net = EvaluationNetwork().to(device)
optimization_net = OptimizationNetwork().to(device)

# 创建优化器
learning_rate = 1e-3
optimizers = {
    'sampling': torch.optim.Adam(sampling_net.parameters(), lr=learning_rate),
    'evaluation': torch.optim.Adam(evaluation_net.parameters(), lr=learning_rate),
    'optimization': torch.optim.Adam(optimization_net.parameters(), lr=learning_rate)
}

### 3.2 训练模型

In [ ]:
num_epochs = 100
save_dir = 'results/models'
os.makedirs(save_dir, exist_ok=True)

# 训练循环
for epoch in range(num_epochs):
    # 训练模式
    sampling_net.train()
    evaluation_net.train()
    optimization_net.train()
    
    train_loss = {'sampling': 0.0, 'evaluation': 0.0, 'optimization': 0.0}
    
    with tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}') as pbar:
        for batch in pbar:
            # 将数据移到设备上
            env_state = batch['env_state'].to(device)
            start = batch['start'].to(device)
            goal = batch['goal'].to(device)
            path = batch['path'].to(device)
            
            # 训练采样网络
            optimizers['sampling'].zero_grad()
            sample_loss = sampling_net.compute_loss(env_state, start, goal)
            sample_loss.backward()
            optimizers['sampling'].step()
            
            # 训练评估网络
            optimizers['evaluation'].zero_grad()
            eval_loss = evaluation_net.compute_loss(env_state, path)
            eval_loss.backward()
            optimizers['evaluation'].step()
            
            # 训练优化网络
            optimizers['optimization'].zero_grad()
            opt_loss = optimization_net.compute_loss(env_state, path)
            opt_loss.backward()
            optimizers['optimization'].step()
            
            # 更新损失
            train_loss['sampling'] += sample_loss.item()
            train_loss['evaluation'] += eval_loss.item()
            train_loss['optimization'] += opt_loss.item()
            
            # 更新进度条
            pbar.set_postfix({
                'sample_loss': sample_loss.item(),
                'eval_loss': eval_loss.item(),
                'opt_loss': opt_loss.item()
            })
    
    # 打印每个 epoch 的平均损失
    print(f"\nEpoch {epoch+1} 平均损失:")
    for key, value in train_loss.items():
        print(f"  {key}: {value/len(train_loader):.4f}")
    
    # 保存模型
    if (epoch + 1) % 10 == 0:
        torch.save(sampling_net.state_dict(), f"{save_dir}/sampling_net_epoch_{epoch+1}.pt")
        torch.save(evaluation_net.state_dict(), f"{save_dir}/evaluation_net_epoch_{epoch+1}.pt")
        torch.save(optimization_net.state_dict(), f"{save_dir}/optimization_net_epoch_{epoch+1}.pt")

## 4. 使用训练好的模型进行路径规划

### 4.1 创建测试环境

In [ ]:
# 创建测试环境
test_env = Environment(width=1000.0, height=500.0)

# 添加一些障碍物
test_env.add_obstacle(x=300, y=250, obstacle_type="circle", radius=50)
test_env.add_obstacle(x=600, y=300, obstacle_type="rectangle", width=100, height=80)
test_env.add_obstacle(x=800, y=200, obstacle_type="circle", radius=40)

# 设置起点和终点
start_point = (100, 100)
goal_point = (900, 400)

# 可视化环境
test_env.visualize()

### 4.2 使用神经网络增强的 RRT 进行路径规划

In [ ]:
# 创建神经网络增强的 RRT 规划器
neural_rrt = NeuralRRT(
    sampling_net=sampling_net,
    evaluation_net=evaluation_net,
    optimization_net=optimization_net,
    start=start_point,
    goal=goal_point,
    env=test_env,
    max_iterations=2000
)

# 进行路径规划
path = neural_rrt.plan()

if path:
    print("路径规划成功!")
    # 可视化规划结果
    test_env.visualize_path(path)
else:
    print("未找到有效路径")

### 4.3 与传统 RRT* 算法对比

In [ ]:
# 使用传统 RRT* 进行规划
rrt_star = RRTStar(
    start=start_point,
    goal=goal_point,
    env=test_env,
    max_iterations=2000
)

rrt_path = rrt_star.plan()

if rrt_path:
    print("RRT* 规划成功!")
    # 可视化规划结果
    test_env.visualize_path(rrt_path)
else:
    print("RRT* 未找到有效路径")

# 比较路径质量
if path and rrt_path:
    neural_length = sum(np.linalg.norm(np.array(path[i+1]) - np.array(path[i])) 
                       for i in range(len(path)-1))
    rrt_length = sum(np.linalg.norm(np.array(rrt_path[i+1]) - np.array(rrt_path[i])) 
                     for i in range(len(rrt_path)-1))
    
    print(f"\n路径长度对比:")
    print(f"神经网络增强 RRT: {neural_length:.2f}")
    print(f"传统 RRT*: {rrt_length:.2f}")
    print(f"改进比例: {((rrt_length - neural_length) / rrt_length * 100):.2f}%")

## 5. 总结

本教程展示了如何：

1. 准备训练数据
2. 训练神经网络模型
3. 使用训练好的模型进行路径规划
4. 与传统 RRT* 算法进行对比

神经网络增强的 RRT 算法通过学习环境特征和历史规划经验，可以：

- 生成更智能的采样策略
- 更准确地评估路径质量
- 优化生成的路径

这些改进使得算法能够更快地找到更优质的路径。